In [59]:
from ebooklib import epub
import ebooklib
from bs4 import BeautifulSoup
import lxml
import KnowledgeDAO as kd
import re

In [93]:
def read_epub(file_path):
    book = epub.read_epub(file_path)
    
    result = []
    for item in book.get_items():
        if item.get_type() == ebooklib.ITEM_DOCUMENT:
            content = item.get_content().decode('utf-8')
            # 处理文档内容，例如打印到控制台
            result.append(content)
    
    return result

#解析知乎爬虫数据，找到超过10赞的回答，若所有回答的赞同数小于10，则找最高赞回答

def get_zhihu_QA(data,agreement_num = 10):
    soup = BeautifulSoup(data[0], 'html.parser')
    h1_tag = soup.find('h1')
    question = h1_tag.text.strip()
    
    answer_list = []
    agreement_list = []
    
    for i in range(1,len(data)):
        soup1 = BeautifulSoup(data[i], 'html.parser') #html.parser
        
        agreement = soup1.find('p', string=re.compile("赞同数："))        
        if agreement:
            agreement_text = agreement.text.split('：')[1].strip()
            if "万" in agreement_text:
                # 去除空格和万字符
                agreement_text = agreement_text.replace(" ", "").replace("万", "")

                # 将字符串转换为浮点数
                number_float = float(agreement_text)

                # 将浮点数乘以一万并转换为整数
                agreement_text = int(number_float * 10000)
        else:
            agreement_text = 0
        agreement_list.append(int(agreement_text))
            
        if int(agreement_text) > agreement_num:
            p_tags = soup1.find_all(['p','li'])
            content = ""
            for p_tag in p_tags:
                if 'data-pid'  in p_tag.attrs:
                    text_p = p_tag.text.strip()
                    content += text_p
            
            answer_list.append(content)
        else:
            continue
    
    if max(agreement_list) <= agreement_num:
        num = agreement_list.index(max(agreement_list))
        soup_max = BeautifulSoup(data[num], 'html.parser')
        p_tags_max = soup_max.find_all(['p','li'])
        content_max = ""
        for p_tag in p_tags_max:
            if 'data-pid'  in p_tag.attrs:
                text_max = p_tag.text.strip()
                content_max += text_max

        answer_list.append(content_max)
        
    return question,answer_list



In [126]:
# 示例使用
folder_path = 'D:/GPT/Data/Zhihu/Quant_QA/data'
epub_file_list = kd.get_file_names(folder_path) # EPUB文件路径

qa = []
# 读取EPUB文件
for i in range(len(epub_file_list)):
    data = read_epub(folder_path+'/'+epub_file_list[i])
    question,answer_list = get_zhihu_QA(data,agreement_num = 10)
    
    if answer_list == ['']:
        continue
    else:
        qa_dic = {
            "Question":question[5:],
            **{f"Answer_{num+1}": answer for num, answer in enumerate(answer_list)}
        }
        qa.append(qa_dic)

In [129]:
kd.save_json(qa,"Zhihu_Quant_QA.json")
kd.save_jsonl(qa,"Zhihu_Quant_QA.jsonl")

In [130]:
size = kd.count_chinese_chars("Zhihu_Quant_QA.jsonl")
#sizes = kd.count_chinese_chars_in_string(qa)
print(size)

2625325


In [90]:
question[5:]

'2023年个人做量化交易靠谱吗？'

In [127]:
print(len(qa))

365


In [134]:
print(qa[1])

{'Question': '2023年个人做量化交易靠谱吗？', 'Answer_1': 'chatgpt都出来了，你觉得量化还有前途？做辅助还行，单纯的量化早晚被ai淘汰。不会长久的。不要相信什么量化冠军，稳定盈利，一个系统吃一辈子的话了。'}
